<a href="https://colab.research.google.com/github/gyanasluitel/Music_Genre_Classification/blob/main/preparing_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as npm

In [2]:
import librosa

In [3]:
import math
import glob

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
path = '/content/drive/My Drive/Sub_Data'
path

'/content/drive/My Drive/Sub_Data'

In [6]:
os.listdir(path)

['images_original', 'genres_original']

In [7]:
DATASET_PATH = os.path.join(path, 'genres_original')
DATASET_PATH

'/content/drive/My Drive/Sub_Data/genres_original'

In [23]:
def prepare_dataset(datapath,n_mfcc=13, n_fft=248,hop_length=512, num_segments=5):
    """
    extracting mfcc features from audio 
    """
    SAMPLE_RATE = 22050
    DURATION = 30
    SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

    #preparing header for dataframe columns
    header = ''
    for i in range(1, 14):
        header += f' mfcc{i}'
    header = header.split()
    header
    
    final_dataframe = pd.DataFrame()
    num_samples_per_segment =int(SAMPLES_PER_TRACK  / num_segments)
    expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)
    
    for dirpath, dirnames, filenames in os.walk(DATASET_PATH):
          if dirpath is not DATASET_PATH:
            dirpath_components = dirpath.split('/')
            semantic_label = dirpath_components[-1]
            # print(f"Dirpath Components: {dirpath_components}")
            # print(f'Semantic Label: {semantic_label}')
            print(f"\nProcessing: {semantic_label}")

            single_file_dataframe = pd.DataFrame()
            #processing files of a specific genre
            for file in filenames:
                  file_path = os.path.join(dirpath,file)
                  # print(f"File Path: {file_path}")
                  label = file_path.split('/')[-2]
                  # print(f"Label: {label}")


                  signal, sr = librosa.load(file_path, sr = SAMPLE_RATE)
                  #process segments extracting mfcc and storing data
                  for s in range(num_segments):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment

                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                                sr = sr,
                                                n_mfcc = n_mfcc,
                                                n_fft = n_fft,
                                                hop_length = hop_length)
                    mfcc = mfcc.T

                    #store mfcc for segment if it has the expected length
                    if len(mfcc) == expected_num_mfcc_vectors_per_segment:
                        segment_dataframe = pd.DataFrame(data = mfcc,
                                    columns =header, 
                                    index = range(len(mfcc)))
                        segment_dataframe['Labels'] = [label] * mfcc.shape[0]
                        
                        #concatenating dataframe of segement into the single file dataframe
                        single_file_dataframe = pd.concat([single_file_dataframe, segment_dataframe],
                                                    axis=0,
                                                    sort=False,
                                                    ignore_index = True)

            final_dataframe  = pd.concat([final_dataframe, single_file_dataframe],
                              axis=0,
                              sort=False,
                              ignore_index = True)  
    return final_dataframe




In [24]:
df = prepare_dataset(DATASET_PATH)


Processing: metal

Processing: rock

Processing: hiphop


/usr/local/lib/python3.7/dist-packages/librosa/filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "



Processing: pop

Processing: jazz

Processing: blues

Processing: disco

Processing: reggae

Processing: classical

Processing: country


In [25]:
df['Labels'].unique()

array(['metal', 'rock', 'hiphop', 'pop', 'jazz', 'blues', 'disco',
       'reggae', 'classical', 'country'], dtype=object)

In [26]:
df

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,Labels
0,-364.062531,-90.791420,-35.297005,48.698944,3.605005,-15.810954,3.845045,17.147272,8.175171,-14.733370,-7.346831,-1.280292,-4.485435,metal
1,-357.282715,-70.076050,-48.885040,32.132942,8.981075,-9.389668,-6.507328,-5.161431,-1.187411,0.277049,-5.940810,-11.849247,-10.966181,metal
2,-362.335938,-67.595856,-52.160065,55.153919,24.046984,-4.233691,-22.290623,1.690348,2.187696,5.058212,-14.868209,-19.218102,-12.790705,metal
3,-355.013306,-78.741760,-35.749840,59.118912,31.603359,-17.079290,-12.175348,-7.910367,-11.798893,-1.861861,-13.079284,-8.189814,-4.452888,metal
4,-345.670349,-66.501671,-46.605743,55.364136,26.576965,-18.905203,-15.367243,-12.026799,-10.002959,-10.082777,2.154212,-8.541779,-8.565044,metal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25895,-345.958771,-14.447283,-63.779541,55.966660,24.309952,16.151093,-18.468191,-19.775593,4.252324,7.497634,-9.175560,-14.709491,-2.870118,country
25896,-346.409424,0.373980,-62.504108,43.894501,16.758186,5.984900,-19.925812,-11.598152,15.054453,19.983093,-15.730133,-17.684364,-2.907649,country
25897,-321.539917,-11.080021,-51.809074,18.817968,29.352173,1.226519,-10.619257,-7.000164,-2.940631,15.152529,-11.508370,-15.230257,-9.451839,country
25898,-313.833099,-44.783813,-40.581181,37.503029,25.615997,-4.249417,-10.102406,-12.080114,-3.075260,16.356224,-11.625240,-7.636725,4.098425,country
